Carga y exploración inicial del dataset SIO-Carnes 

https://www.magyp.gob.ar/sitio/areas/ss_mercados_agropecuarios/sistemas-de-informacion-de-operaciones-sio/sio-carnes/index.php

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/completo-ago20-ago21.csv')
print(f"Shape: {df.shape}")
df.info()

/tmp/ipykernel_67177/3564064497.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/completo-ago20-ago21.csv')


Shape: (1721218, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721218 entries, 0 to 1721217
Data columns (total 11 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Fecha Comprobante        object 
 1   Provincia Origen         object 
 2   Partido Origen           object 
 3   Zona Destino             object 
 4   Raza                     object 
 5   Categoria                object 
 6   Cabezas Comercializadas  float64
 7   Unidad de Medida         object 
 8   Precio cabeza            object 
 9   Precio Kg                object 
 10  Cantidad de Kg           float64
dtypes: float64(2), object(9)
memory usage: 144.5+ MB


Estadísticas descriptivas de las variables numéricas principales.

In [2]:
df.describe()

,Cabezas Comercializadas,Cantidad de Kg
count,1.721212e+06,1.627932e+06
mean,5.726315e+01,4.962836e+03
std,5.353858e+04,1.224111e+04
min,1.000000e+00,1.000000e+00
25%,2.000000e+00,8.300000e+02
50%,6.000000e+00,2.380000e+03
75%,1.700000e+01,6.660000e+03
max,7.022330e+07,6.781100e+06


Primeros registros del dataset para visualizar la estructura de los datos.

In [3]:
df.head(10)

,Fecha Comprobante,Provincia Origen,Partido Origen,Zona Destino,Raza,Categoria,Cabezas Comercializadas,Unidad de Medida,Precio cabeza,Precio Kg,Cantidad de Kg
0,23/08/2021,BUENOS AIRES,NUEVE DE JULIO,ZONA 3 B,Aberdeen Angus,Bovino Vaquillona Especial Liviana hasta 390 k...,7.0,Kg. Vivo,NaN,169,1970.0
1,23/08/2021,BUENOS AIRES,NUEVE DE JULIO,ZONA 3 B,Aberdeen Angus,Bovino Novillito Especial Liviano hasta 390 ki...,3.0,Kg. Vivo,NaN,169,910.0
2,23/08/2021,BUENOS AIRES,NUEVE DE JULIO,ZONA 3 B,Aberdeen Angus,Bovino Novillito Especial Pesado más de 390 ki...,1.0,Kg. Vivo,NaN,158,395.0
3,22/08/2021,CORDOBA,JUAREZ CELMAN,ZONA 5 B,Aberdeen Angus,Bovino Vaquillona Regular (4 dientes),14.0,Kg. Vivo,NaN,154,4242.0
4,22/08/2021,CORDOBA,JUAREZ CELMAN,ZONA 5 B,Aberdeen Angus,Bovino Novillito Regular (4 dientes),53.0,Kg. Vivo,NaN,154,16059.0
5,22/08/2021,ENTRE RIOS,FEDERAL,ZONA 5 B,Otra 99,Bovino Vaca Regular (6 o más dientes),5.0,Kg. Vivo,NaN,100,1970.0
6,22/08/2021,ENTRE RIOS,FEDERAL,ZONA 5 B,Otra 99,Bovino Toro Regular (más de 2 dientes),4.0,Kg. Vivo,NaN,100,2500.0
7,22/08/2021,SANTA FE,SAN MARTIN,ZONA 5 B,Holando Argentino,Bovino Novillito Regular (4 dientes),23.0,Kg. Vivo,NaN,114,8573.0
8,22/08/2021,ENTRE RIOS,FEDERACION,ZONA 5 B,Otra 99,Bovino Novillito Regular (4 dientes),5.0,Kg. Vivo,NaN,100,1950.0
9,22/08/2021,BUENOS AIRES,VEINTICINCO DE MAYO,ZONA 3 B,Aberdeen Angus,Bubalino Ternera,18.0,Kg. Vivo,NaN,160,6449.0


Análisis de valores faltantes por columna.

In [4]:
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (valores_faltantes / len(df) * 100).round(2)
pd.DataFrame({
    'Valores_Faltantes': valores_faltantes,
    'Porcentaje': porcentaje_faltantes
}).sort_values('Porcentaje', ascending=False)


,Valores_Faltantes,Porcentaje
Precio cabeza,1627938,94.58
Precio Kg,93286,5.42
Cantidad de Kg,93286,5.42
Partido Origen,6,0.00
Provincia Origen,6,0.00
Fecha Comprobante,6,0.00
Zona Destino,6,0.00
Cabezas Comercializadas,6,0.00
Categoria,6,0.00
Raza,6,0.00


Ejemplos específicos de outliers extremos en Cabezas Comercializadas.

In [5]:
df_numeric = df[df['Cabezas Comercializadas'].notna()]
df_numeric['Cabezas Comercializadas'] = pd.to_numeric(df_numeric['Cabezas Comercializadas'], errors='coerce')

Q1 = df_numeric['Cabezas Comercializadas'].quantile(0.25)
Q3 = df_numeric['Cabezas Comercializadas'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Límite superior outliers: {limite_superior}")

outliers_cabezas = df_numeric[df_numeric['Cabezas Comercializadas'] > limite_superior]
print(f"\nOutliers encontrados: {len(outliers_cabezas)}")

outliers_cabezas[['Cabezas Comercializadas', 'Cantidad de Kg', 'Precio Kg', 'Provincia Origen', 'Raza']].head(20)


Q1: 2.0, Q3: 17.0, IQR: 15.0
Límite superior outliers: 39.5

Outliers encontrados: 124732


/tmp/ipykernel_67177/1797092803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['Cabezas Comercializadas'] = pd.to_numeric(df_numeric['Cabezas Comercializadas'], errors='coerce')


,Cabezas Comercializadas,Cantidad de Kg,Precio Kg,Provincia Origen,Raza
4,53.0,16059.0,154,CORDOBA,Aberdeen Angus
16,55.0,18200.0,95,FORMOSA,Brahman
17,43.0,18560.0,187,BUENOS AIRES,Otra 99
18,43.0,18710.0,187,BUENOS AIRES,Otra 99
19,43.0,18710.0,187,BUENOS AIRES,Otra 99
28,55.0,18200.0,95,FORMOSA,Brahman
32,43.0,18560.0,187,BUENOS AIRES,Otra 99
38,51.0,19540.0,"129,01",SANTA FE,Otra 99
45,41.0,17410.0,152,BUENOS AIRES,Aberdeen Angus
46,43.0,43.0,"56959,7",SAN LUIS,Aberdeen Angus


Casos extremos de outliers que superan valores verosimilmente posibles.

In [6]:
casos_extremos = df_numeric[df_numeric['Cabezas Comercializadas'] > 100000]
print(f"Casos con más de 100,000 cabezas: {len(casos_extremos)}")
casos_extremos[['Cabezas Comercializadas', 'Cantidad de Kg', 'Precio Kg', 'Provincia Origen']].head(10)

Casos con más de 100,000 cabezas: 10


,Cabezas Comercializadas,Cantidad de Kg,Precio Kg,Provincia Origen
207019,127024.0,3263.0,140,CORDOBA
242655,70223300.0,4091.0,"197,383",BUENOS AIRES
328555,828000.0,2640.0,85,ENTRE RIOS
350580,136083.0,3200.0,"139,7",BUENOS AIRES
805332,603028.0,3225.0,120,BUENOS AIRES
807500,603028.0,3225.0,120,BUENOS AIRES
1024828,165120.0,5120.0,80,BUENOS AIRES
1104657,226003.0,5500.0,100,CORDOBA
1207255,535012.0,821.0,"82,518",BUENOS AIRES
1310475,656343.0,8200.0,90,BUENOS AIRES


- Crear discretización de variables continuas. Mantener los outlier con una categoria. 

Outliers en Precio por Kg que sugieren errores de sistema.

In [7]:
df_precio = df[df['Precio Kg'].notna()].copy()
df_precio['Precio Kg'] = pd.to_numeric(df_precio['Precio Kg'], errors='coerce')
df_precio = df_precio[df_precio['Precio Kg'].notna()]

Q1_precio = df_precio['Precio Kg'].quantile(0.25)
Q3_precio = df_precio['Precio Kg'].quantile(0.75)
IQR_precio = Q3_precio - Q1_precio
limite_inf_precio = Q1_precio - 1.5 * IQR_precio
limite_sup_precio = Q3_precio + 1.5 * IQR_precio

print(f"Límites precio: [{limite_inf_precio:.2f}, {limite_sup_precio:.2f}]")

outliers_precio = df_precio[(df_precio['Precio Kg'] < limite_inf_precio) | (df_precio['Precio Kg'] > limite_sup_precio)]
print(f"Outliers en precio: {len(outliers_precio)}")

outliers_precio[['Precio Kg', 'Cantidad de Kg', 'Cabezas Comercializadas', 'Provincia Origen']].head(20)


Límites precio: [17.50, 197.50]
Outliers en precio: 19028


,Precio Kg,Cantidad de Kg,Cabezas Comercializadas,Provincia Origen
343,200.0,3330.0,10.0,CIUDAD AUTONOMA DE BUENOS AIRES
376,200.0,5090.0,15.0,CIUDAD AUTONOMA DE BUENOS AIRES
377,205.0,4860.0,15.0,CIUDAD AUTONOMA DE BUENOS AIRES
421,200.0,4150.0,13.0,CIUDAD AUTONOMA DE BUENOS AIRES
504,206.0,12517.0,38.0,SANTA FE
505,206.0,2303.0,7.0,SANTA FE
621,200.0,8640.0,27.0,CIUDAD AUTONOMA DE BUENOS AIRES
655,220.0,4288.0,13.0,NEUQUEN
718,40880.0,1.0,1.0,LA PAMPA
733,36000.0,3.0,3.0,LA PAMPA


Análisis de registros duplicados en el dataset.

In [12]:
duplicados = df.duplicated().sum()
porcentaje_duplicados = (duplicados / len(df) * 100).round(2)
print(f"Total duplicados: {duplicados} ({porcentaje_duplicados}%)")

Total duplicados: 799789 (46.47%)


Ejemplos concretos de registros duplicados para verificar si son errores o transacciones legítimas.

In [13]:
df_duplicados = df[df.duplicated(keep=False)].sort_values(df.columns.tolist())

In [14]:
primer_grupo_duplicado = df_duplicados.iloc[:4]
primer_grupo_duplicado

,Fecha Comprobante,Provincia Origen,Partido Origen,Zona Destino,Raza,Categoria,Cabezas Comercializadas,Unidad de Medida,Precio cabeza,Precio Kg,Cantidad de Kg
930768,01/01/2021,BUENOS AIRES,ARRECIFES,ZONA 2 B,Bovino Criollo,Bovino MEJ Regular (hasta 2 dientes),34.0,Kg. Vivo,NaN,145,12920.0
930821,01/01/2021,BUENOS AIRES,ARRECIFES,ZONA 2 B,Bovino Criollo,Bovino MEJ Regular (hasta 2 dientes),34.0,Kg. Vivo,NaN,145,12920.0
930727,01/01/2021,BUENOS AIRES,BENITO JUAREZ,ZONA 4 B,Aberdeen Angus,Bovino Novillito Regular (4 dientes),12.0,Kg. Vivo,NaN,117,4912.0
930767,01/01/2021,BUENOS AIRES,BENITO JUAREZ,ZONA 4 B,Aberdeen Angus,Bovino Novillito Regular (4 dientes),12.0,Kg. Vivo,NaN,117,4912.0


Conteo de cuántas veces se repite cada registro duplicado.

In [16]:
df_duplicados.head(10)

,Fecha Comprobante,Provincia Origen,Partido Origen,Zona Destino,Raza,Categoria,Cabezas Comercializadas,Unidad de Medida,Precio cabeza,Precio Kg,Cantidad de Kg
930768,01/01/2021,BUENOS AIRES,ARRECIFES,ZONA 2 B,Bovino Criollo,Bovino MEJ Regular (hasta 2 dientes),34.0,Kg. Vivo,NaN,145,12920.0
930821,01/01/2021,BUENOS AIRES,ARRECIFES,ZONA 2 B,Bovino Criollo,Bovino MEJ Regular (hasta 2 dientes),34.0,Kg. Vivo,NaN,145,12920.0
930727,01/01/2021,BUENOS AIRES,BENITO JUAREZ,ZONA 4 B,Aberdeen Angus,Bovino Novillito Regular (4 dientes),12.0,Kg. Vivo,NaN,117,4912.0
930767,01/01/2021,BUENOS AIRES,BENITO JUAREZ,ZONA 4 B,Aberdeen Angus,Bovino Novillito Regular (4 dientes),12.0,Kg. Vivo,NaN,117,4912.0
930895,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),4.0,Kg. Vivo,NaN,120,1560.0
930897,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),4.0,Kg. Vivo,NaN,120,1560.0
930901,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),4.0,Kg. Vivo,NaN,120,1560.0
930915,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),4.0,Kg. Vivo,NaN,120,1560.0
930885,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),8.0,Kg. Vivo,NaN,120,3120.0
930898,01/01/2021,BUENOS AIRES,LOBERIA,ZONA 4 B,Jersey,Bovino Novillito Regular (4 dientes),8.0,Kg. Vivo,NaN,120,3120.0


In [17]:
df_duplicados.shape


(1245155, 11)

Inconsistencias peso-cantidad que sugieren errores de datos.

In [18]:
df_peso = df.copy()
df_peso['Cabezas Comercializadas'] = pd.to_numeric(df_peso['Cabezas Comercializadas'], errors='coerce')
df_peso['Cantidad de Kg'] = pd.to_numeric(df_peso['Cantidad de Kg'], errors='coerce')

df_peso = df_peso[(df_peso['Cabezas Comercializadas'].notna()) & (df_peso['Cantidad de Kg'].notna())]
df_peso = df_peso[(df_peso['Cabezas Comercializadas'] > 0) & (df_peso['Cantidad de Kg'] > 0)]

df_peso['Peso_por_Cabeza'] = df_peso['Cantidad de Kg'] / df_peso['Cabezas Comercializadas']

casos_imposibles = df_peso[(df_peso['Peso_por_Cabeza'] < 50) | (df_peso['Peso_por_Cabeza'] > 2000)]
print(f"Casos con peso por cabeza biológicamente imposible: {len(casos_imposibles)}")

casos_imposibles[['Cabezas Comercializadas', 'Cantidad de Kg', 'Peso_por_Cabeza', 'Raza', 'Categoria']].head(15)


Casos con peso por cabeza biológicamente imposible: 5917


,Cabezas Comercializadas,Cantidad de Kg,Peso_por_Cabeza,Raza,Categoria
46,43.0,43.0,1.000000,Aberdeen Angus,Bovino Vaca Regular (6 o más dientes)
63,43.0,43.0,1.000000,Aberdeen Angus,Bovino Vaca Regular (6 o más dientes)
124,1.0,2055.0,2055.000000,Aberdeen Angus,Bovino Vaquillona Especial Liviana hasta 390 k...
166,11.0,11.0,1.000000,Aberdeen Angus,Bovino Vaca Regular (6 o más dientes)
672,11.0,11.0,1.000000,Aberdeen Angus,Bovino Vaca Regular (6 o más dientes)
718,1.0,1.0,1.000000,Aberdeen Angus,Bovino MEJ Regular (hasta 2 dientes)
719,3.0,3.0,1.000000,Aberdeen Angus,Bovino Vaca Regular (6 o más dientes)
732,13.0,13.0,1.000000,Aberdeen Angus,Bovino Toro Regular (más de 2 dientes)
733,3.0,3.0,1.000000,Aberdeen Angus,Bovino MEJ Regular (hasta 2 dientes)
738,2.0,1.0,0.500000,Otra 99,Bovino Vaca Conserva Buena (6 o más dientes)


# Pipeline de limpieza de datos

FLUJO DE ELIMINACIÓN:
Dataset original: 1.72M registros
  - (Eliminación columna "Precio cabeza")
  - (Eliminación de 93286 registros con valores faltantes)
Dataset antes de eliminar duplicados: 1627932 registros
  - (Eliminación de duplicados)
Dataset final: 877899 registros

EJEMPLO PRÁCTICO DE DUPLICACIÓN

REGISTRO ORIGINAL:
23/08/2021, BUENOS AIRES, NUEVE DE JULIO, ZONA 3 B, Aberdeen Angus, 
Bovino Vaquillona, 7, Kg. Vivo, 169, 1970

REGISTRO DUPLICADO (TODAS las columnas iguales):
23/08/2021, BUENOS AIRES, NUEVE DE JULIO, ZONA 3 B, Aberdeen Angus, 
Bovino Vaquillona, 7, Kg. Vivo, 169, 1970

NO DUPLICADO (Precio Kg diferente):
23/08/2021, BUENOS AIRES, NUEVE DE JULIO, ZONA 3 B, Aberdeen Angus, 
Bovino Vaquillona, 7, Kg. Vivo, 170, 1970


HIPÓTESIS SOBRE EL ORIGEN DE DUPLICADOS:
- Errores en la captura de datos del sistema SIO-Carnes
- Re-envíos de la misma transacción por problemas técnicos o multiples fuentes informantes:
   - "El sistema se basa en información suministrada por AFIP a través de las liquidaciones electrónicas de compra venta de ganado bovino y porcino con destino a faena, emitidas por consignatarios de hacienda, establecimientos faenadores o usuarios y productores."
- Sistemas múltiples reportando la misma operación
- Problemas de sincronización entre bases de datos AFIP-SENASA
- Duplicación durante procesos de migración de datos

- peso promedio por cabeza. 
- Si tuvieras estas reglas con este soporte, este lift y esta confianza, qué reglas eligirías y porque?